In [1]:
import numpy as np
import sympy as sp

import random
import math

import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

# 2

*ПЕРЕСЕЧЕНИЕ ОТРЕЗКОВ ПРЯМЫХ МЕТОДОМ SWEEPING LINE*

**Применение метода заметания плоскости для поиска пересечений отрезков**

Для задачи поиска всех пар пересекающихся отрезков на плоскости, метод заметания плоскости работает следующим образом:

1. Инициализация: Создаем список событий, которые включают начальные и конечные точки всех отрезков. Сортируем эти события по координате x.

2. Обработка событий: Используем структуру данных для хранения отрезков, которые пересекают текущую вертикальную линию. При обработке каждого события:

    - Начало отрезка: Добавляем отрезок в структуру данных. Проверяем, пересекается ли он с отрезками, находящимися выше и ниже него в структуре.
    - Конец отрезка: Удаляем отрезок из структуры данных.
    - Пересечение: Если найдено пересечение, добавляем его в список пересечений.


**Алгоритм нахождения всех пар пересекающихся отрезков**

1. Сортировка событий: Сортируем все точки (начальные и конечные) отрезков по координате x. Если координаты x совпадают, сортируем по координате y.

2. Инициализация структуры данных: Создаем структуру данных для хранения отрезков, пересекающих текущую вертикальную линию.

3. Обработка событий:

    - Для каждой точки:

        - Если это начало отрезка, добавляем отрезок в структуру данных и проверяем пересечения с соседними отрезками.

        - Если это конец отрезка, удаляем отрезок из структуры данных.

        - Если найдено пересечение, добавляем его в список пересечений.

4. Вывод результатов: Возвращаем список всех найденных пересечений.

**Спроектируйте функции**

1. sort_events(segments):

    - Входные аргументы: segments — список отрезков.

    - Что делает: Сортирует все точки (начальные и конечные) отрезков по координате x.

    - Выход: Отсортированный список событий.

2. find_intersections(segments):

    - Входные аргументы: segments — список отрезков.

    - Что делает: Находит все пары пересекающихся отрезков, используя метод заметания плоскости.

    - Выход: Список пар пересекающихся отрезков.

3. process_event(event, active_segments, intersections):

    - Входные аргументы: event — текущее событие, active_segments — структура данных с активными отрезками, intersections — список пересечений.

    - Что делает: Обрабатывает текущее событие (начало/конец отрезка или пересечение).

    - Выход: Обновленные active_segments и intersections.

4. check_intersection(segment1, segment2):

    - Входные аргументы: segment1, segment2 — два отрезка.

    - Что делает: Проверяет, пересекаются ли два отрезка.

    - Выход: True, если отрезки пересекаются, иначе False.

ссылка на диаграмму

https://lucid.app/lucidchart/17e869e3-4338-49ac-97e8-b0eb1892c227/edit?viewport_loc=-662%2C-756%2C3803%2C1929%2C0_0&invitationId=inv_184e8c7b-c5b3-4671-a6b3-b9c7132be9a1

In [2]:
from sortedcontainers import SortedList

In [3]:
class Segment:
    def __init__(self, start, end):
        self.start = start
        self.end = end

def sort_events(segments):
    events = []
    for segment in segments:
        events.append((segment.start, 'start', segment))
        events.append((segment.end, 'end', segment))
    events.sort(key=lambda x: (x[0][0], x[0][1]))
    return events

def check_intersection(segment1, segment2):
    def orientation(p, q, r):
        val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
        if val == 0:
            return 0
        elif val > 0:
            return 1
        else:
            return 2

    def on_segment(p, q, r):
        if min(p[0], r[0]) <= q[0] <= max(p[0], r[0]) and min(p[1], r[1]) <= q[1] <= max(p[1], r[1]):
            return True
        return False

    p1, q1 = segment1.start, segment1.end
    p2, q2 = segment2.start, segment2.end

    o1 = orientation(p1, q1, p2)
    o2 = orientation(p1, q1, q2)
    o3 = orientation(p2, q2, p1)
    o4 = orientation(p2, q2, q1)

    if o1 != o2 and o3 != o4:
        return True

    if o1 == 0 and on_segment(p1, p2, q1):
        return True
    if o2 == 0 and on_segment(p1, q2, q1):
        return True
    if o3 == 0 and on_segment(p2, p1, q2):
        return True
    if o4 == 0 and on_segment(p2, q1, q2):
        return True

    return False

def process_event(event, active_segments, intersections):
    point, event_type, segment = event
    if event_type == 'start':
        active_segments.add(segment)
        idx = active_segments.index(segment)
        if idx > 0 and check_intersection(segment, active_segments[idx - 1]):
            intersections.add((segment, active_segments[idx - 1]))
        if idx < len(active_segments) - 1 and check_intersection(segment, active_segments[idx + 1]):
            intersections.add((segment, active_segments[idx + 1]))
    elif event_type == 'end':
        if segment in active_segments:
            idx = active_segments.index(segment)
            if idx > 0 and idx < len(active_segments) - 1:
                if check_intersection(active_segments[idx - 1], active_segments[idx + 1]):
                    intersections.add((active_segments[idx - 1], active_segments[idx + 1]))
            active_segments.remove(segment)

def find_intersections(segments):
    events = sort_events(segments)
    active_segments = SortedList(key=lambda x: x.start[1])
    intersections = set()

    for event in events:
        process_event(event, active_segments, intersections)

    return intersections

In [4]:
# Пример использования
segments = [
    Segment((1, 1), (4, 4)),
    Segment((2, 2), (5, 5)),
    Segment((3, 3), (6, 6)),
    Segment((1, 4), (4, 1)),
    Segment((2, 5), (5, 2)),
    Segment((3, 6), (6, 3)),
    Segment((1, 3), (5, -5))
]

intersections = find_intersections(segments)
for intersection in intersections:
    print(f"Пересекаются отрезки: {intersection[0].start}->{intersection[0].end} и {intersection[1].start}->{intersection[1].end}")

Пересекаются отрезки: (3, 3)->(6, 6) и (3, 6)->(6, 3)
Пересекаются отрезки: (2, 2)->(5, 5) и (3, 3)->(6, 6)
Пересекаются отрезки: (3, 3)->(6, 6) и (2, 5)->(5, 2)
Пересекаются отрезки: (1, 3)->(5, -5) и (1, 1)->(4, 4)
Пересекаются отрезки: (2, 2)->(5, 5) и (1, 1)->(4, 4)
